In [1]:
import numpy
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
from six.moves import cPickle
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from keras.datasets import reuters
import pandas as pd
from twitter_parser import twitter_parser
from nltk.tokenize import TweetTokenizer
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import Normalizer

In [4]:
crowdflower = pd.read_csv('./data/Crowdflower/socialmedia-disaster-tweets-DFE.csv', encoding='latin-1')

In [51]:
crowdflower[:10]

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN
5,778243828,True,golden,140,NaN,Relevant,1.0000,Relevant,NaN,NaN,All residents asked to 'shelter in place' are ...,17.0,NaN
6,778243831,True,golden,142,NaN,Relevant,1.0000,Relevant,NaN,NaN,"13,000 people receive #wildfires evacuation or...",18.0,NaN
7,778243832,True,golden,151,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,19.0,NaN
8,778243833,True,golden,143,NaN,Relevant,1.0000,Relevant,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,20.0,NaN
9,778243834,True,golden,136,NaN,Relevant,0.9606,Relevant\nCan't Decide,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,21.0,NaN


In [5]:
from sklearn.model_selection import train_test_split

corpus, docs_new, corpus_target, docs_new_target = train_test_split(crowdflower['text'], crowdflower['choose_one'], test_size=0.33, random_state=42)

In [5]:
corpus = list(crowdflower[:int(len(crowdflower)*0.9)]['text'])
print("Training with {} tweets.".format(len(corpus)))
docs_new = list(crowdflower[:-int(len(crowdflower)*0.9)]['text'])
print("Testing with {} tweets.".format(len(docs_new)))

Testing with 1088 tweets.


In [6]:
import sys  
sys.path.append('.')
import twokenize

In [7]:
tokenize_it = twitter_parser.Tokenizer()
tknzr = TweetTokenizer()
vectorizer = TfidfVectorizer(min_df=1, tokenizer=tknzr.tokenize)
#vectorizer = TfidfVectorizer(min_df=1, tokenizer=tokenize_it.tweet_to_tokens)
#vectorizer_twitter = TfidfVectorizer(min_df=1, )
#X = vectorizer.fit_transform(corpus)

In [8]:
corpus_target = list(crowdflower[:int(len(crowdflower)*0.9)]['choose_one'])
docs_new_target = list(crowdflower[:-int(len(crowdflower)*0.9)]['choose_one'])

In [12]:
X_lsa

NameError: name 'X_lsa' is not defined

In [77]:
clf = MultinomialNB().fit(X_lsa, corpus_target[:100])

ValueError: Input X must be non-negative

In [57]:
clf = RandomForestClassifier().fit(X_lsa, corpus_target)

In [58]:
test_lsa = lsa.transform(lsa_vect.transform(docs_new))
predict_results = clf.predict(test_lsa)

In [59]:
numpy.mean(predict_results == docs_new_target)

0.62646239554317551

['Help', 'me', ',', 'I', 'am', 'trapped', 'under', 'a', 'bridge']

In [217]:
clf.predict(lsa.transform(lsa_vect.transform(["help me I am trapped :("])))

array(['Not Relevant'], 
      dtype='<U12')

In [218]:
from sklearn.metrics import roc_curve, auc
from sklearn import metrics

#various "fitness" metrics
print("Train accuracy: {} \n".format(clf.score(X_lsa, corpus_target)))

#various "fitness" metrics
print("Train accuracy: {} \n".format(clf.score(X_lsa, corpus_target)))
print("Test accuracy: {} \n".format(clf.score(test_lsa, docs_new_target)))
f_score = metrics.f1_score(docs_new_target,predict_results,pos_label='Relevant')
print("F1 score: {} \n".format(f_score))

#confusion matrix
cm = metrics.confusion_matrix(docs_new_target, predict_results)
print("Confusion matrix: \n")
print("-Legend")
print(numpy.array([['True "not disaster"', 'False "disaster"'],['False "not disaster"', 'True "disaster"']]))
print("\n-Prediction")
print(cm)

Train accuracy: 0.9616877809562729 

Train accuracy: 0.9616877809562729 

Test accuracy: 0.9623161764705882 

F1 score: 0.9621120310716971 

Confusion matrix: 

-Legend
[['True "not disaster"' 'False "disaster"']
 ['False "not disaster"' 'True "disaster"']]

-Prediction
[[  1   0   0]
 [  0 665  10]
 [  0  31 381]]


In [9]:
from sklearn import tree
from sklearn import svm

text_clf = Pipeline([('tfidf', vectorizer),
                      #('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))
                      ('clf', svm.LinearSVC(C = 1.0))
                     ,])

Tokenization Example

In [150]:
#numpy.unique(boston[:int(len(boston)*0.9)]['Information Type'])
twokenize.simpleTokenize("Help me, I am trapped under a bridge")

['Help', 'me', ',', 'I', 'am', 'trapped', 'under', 'a', 'bridge']

Setup Train + Test

In [10]:
#boston_X = list(boston[:int(len(boston)*0.9)]['Tweet Text'])
#boston_Y = list(boston[:int(len(boston)*0.9)]['Information Type'])
#text_clf = text_clf.fit(boston_X, boston_Y)
text_clf = text_clf.fit(corpus, corpus_target)

In [11]:
#boston_test_x = list(boston[:-int(len(boston)*0.9)]['Tweet Text'])
#boston_test_y = list(boston[:-int(len(boston)*0.9)]['Information Type'])
#text_clf_predicted = text_clf.predict(boston_test_x)
text_clf_predicted = text_clf.predict(docs_new)

In [177]:
numpy.mean(text_clf_predicted == docs_new_target)

0.79025069637883005

In [178]:
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
import numpy as np
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Ridge
from sklearn.metrics import precision_recall_fscore_support

#various "fitness" metrics
print("Train accuracy: {} \n".format(text_clf.score(corpus, corpus_target)))
print("Test accuracy: {} \n".format(text_clf.score(docs_new, docs_new_target)))
f_score = metrics.f1_score(docs_new_target,text_clf_predicted,pos_label='Relevant', average='weighted')
print("F1 score: {} \n".format(f_score))

#Precision & Recall & Accuracy

#print("Precision: {}; Recall: {}; FScore: {}".format()
print(precision_recall_fscore_support(docs_new_target, text_clf_predicted, average='weighted'))
#train_scores, valid_scores = validation_curve(Ridge(), list(range(1,len(corpus_target)+1)), corpus_target, "alpha", np.logspace(-7, 3, 3))

#confusion matrix
cm = metrics.confusion_matrix(docs_new_target, text_clf_predicted)
print("Confusion matrix: \n")
print("-Legend")
print(numpy.array([['True "not disaster"', 'False "disaster"'],['False "not disaster"', 'True "disaster"']]))
print("\n-Prediction")
print(cm)

Train accuracy: 0.9822948119681582 

Test accuracy: 0.79025069637883 

F1 score: 0.7885995903259091 

(0.78852873864741546, 0.79025069637883005, 0.78859959032590909, None)
Confusion matrix: 

-Legend
[['True "not disaster"' 'False "disaster"']
 ['False "not disaster"' 'True "disaster"']]

-Prediction
[[   0    5    0]
 [   0 1717  319]
 [   0  429 1120]]


/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1023: UserWarning: Note that pos_label (set to 'Relevant') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
import tweepy
crisis_lexicon = pd.read_table('./data/CrisisLexLexicon/CrisisLexRec.txt', header=None)
crisis_lexicon = list(crisis_lexicon[0])

In [13]:
non_relevant = 0
total = 0

In [14]:
import tweepy

consumer_key = 'welp'
consumer_secret = 'welp'
access_token = 'welp'
access_token_secret = 'welp'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

#override tweepy.StreamListener to add logic to on_status
class CrisisStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if not status.retweeted and 'RT @' not in status.text:
            prediction = text_clf.predict([status.text])
            #total += 1
            if prediction[0] == 'Relevant':
                print(text_clf.predict([status.text]),'\t', status.author.screen_name, ':', status.text)
            #else: non_relevant += 1
    def on_error(self, status):
        print(status)

In [15]:
crisisListener = CrisisStreamListener()
crisisStream = tweepy.Stream(auth = api.auth, listener=crisisListener)
crisisStream.filter(track=crisis_lexicon, async=True)

['Relevant'] 	 omahamjf : "Former RB Joe McKnight Killed in Shooting" via @BleacherReport App: https://t.co/LntjxN1kGu
['Relevant'] 	 TheSpaceCEO : Under Armour Unisex Storm Recruit Backpack, Red (601), One Size Reviews - https://t.co/1REghsV2xa
#backpack #luggage https://t.co/dnvecLgZAK
['Relevant'] 	 fa078a2f38c645d : @blanketcrap  that sounds like an illness in itself.  From kidney damage? Dont wish to pry, sorry
['Relevant'] 	 IzumiFany : @bigbangtbh please give receipt of what has been cleared and what is exposing him bcs afaik, case stil going on n yg stil in investigation
['Relevant'] 	 anachheiri : https://t.co/V1FngB4OLB RT https://t.co/nxVnm34ZWm
['Relevant'] 	 JAscariat : @AtticusSays

Fake die like

Fury fake death in TWS for 10mins
Or
Thor fake death in Thor &amp; Thor2 for 5 min
Or
Groo… https://t.co/mNzdTCXIkq
['Relevant'] 	 OZzSue : @DanielAndrewsMP Former high-ranking police officer Ashby tells about terrifying encounter while driving to Geelong… https://t.co/34MYcgK68

In [67]:
results_df = pd.DataFrame({'text': boston_test_x, 'predicted':text_clf_predicted, 'ground_truth':boston_test_y}).sample(100)
pd.options.display.max_colwidth = 100
print(results_df[results_df['ground_truth'] != results_df['predicted']].to_string())

                    ground_truth                   predicted                                                                                                 text
77      Other Useful Information          Caution and advice  RT @cnnbreak: Flooding chaos in Philippine's capital: Flood waters were rising in parts of the P...
36            Caution and advice  Donations and volunteering       RT @themallph: Hey guys! Use the hashtag #RescuePH for rescue operations and flood monitoring!
35      Other Useful Information          Caution and advice  RT @YahooPH: use #rescuePH hashtag to allow all media outlets to monitor tweets related to the f...
84  Infrastructure and utilities  Donations and volunteering  RT @rancyamor: @annecurtissmith please rt. Thanks. Needs rescue!  #rescuePH 6 Monte Carlo St. Ci...


## Validation

In [18]:
import numpy as np
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Ridge
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(docs_new_target, text_clf_predicted, average='weighted')
#train_scores, valid_scores = validation_curve(Ridge(), list(range(1,len(corpus_target)+1)), corpus_target, "alpha", np.logspace(-7, 3, 3))

/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.79048649684762295, 0.79220055710306403, 0.7905934375611483, None)

In [ ]:
len(boston[:int(len(boston)*0.9)])

In [20]:
true_k = numpy.unique(corpus_target).shape[0]

In [21]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=500, 
                         algorithm='randomized',
                         n_iter=10, random_state=42)
svd_model = TruncatedSVD(true_k)

In [152]:
tknzr = TweetTokenizer()
hasher = HashingVectorizer(n_features=2**18,
                           stop_words='english', non_negative=True,
                           norm=None, binary=False, tokenizer=tknzr.tokenize)

In [153]:
from sklearn.pipeline import make_pipeline
#text_clf = Pipeline([('tfidf', vectorizer),
#                      ('svd', svd_model)])
lsa_vect = make_pipeline(hasher,TfidfTransformer())

In [55]:
X = lsa_vect.fit_transform(corpus)
#clf = MultinomialNB().fit(X, corpus_target)

In [56]:
from sklearn.pipeline import make_pipeline
lsa = make_pipeline(svd_model, Normalizer(copy=False))
X_lsa = lsa.fit_transform(X)

In [45]:
from sklearn import svm
cl_svm = svm.SVC(gamma=0.001, C=100.)
cl_svm.fit(X_lsa, corpus_target)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
numpy.mean(cl_svm.predict(lsa.transform(lsa_vect.transform(docs_new))) == docs_new_target)

0.63231197771587744

# Taking a crack using TF

In [8]:
numpy.max(corpus_target)

TypeError: cannot perform reduce with flexible type

In [47]:
from sklearn.preprocessing import LabelEncoder
lbl_encoder = LabelEncoder()
lbl_encoder.fit(corpus_target)
encoded_Y = lbl_encoder.transform(corpus_target)
encoded_test_Y = lbl_encoder.transform(docs_new_target)
nb_classes = numpy.max(encoded_Y)+1
print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
Y_train = np_utils.to_categorical(encoded_Y, nb_classes)
Y_test = np_utils.to_categorical(encoded_test_Y, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)
#tokenize_it.tweet_to_tokens

Convert class vector to binary class matrix (for use with categorical_crossentropy)
Y_train shape: (7286, 3)
Y_test shape: (3590, 3)


In [15]:
lbl_encoder.inverse_transform(2)

'Relevant'

In [22]:
numpy.max(list(map(lambda x: len(x), sequences)))

32

In [43]:
from keras.preprocessing.sequence import pad_sequences
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None, lower=True, split=" ")
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQUENCE_LENGTH = numpy.max(list(map(lambda x: len(x), sequences)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
indices = numpy.arange(data.shape[0])
numpy.random.shuffle(indices)
data = data[indices]

word_index = tokenizer.word_index
EMBEDDING_DIM = 300
embedding_matrix = numpy.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [44]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
#embedding_layer = Embedding(len(word_index) + 1,
#                            EMBEDDING_DIM,
#                            input_length=MAX_SEQUENCE_LENGTH)

In [50]:
from keras.layers.core    import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D

dropout_ratio = 0.5
nb_filter=150
filter_length=3
pool_length=3
nb_classes = 3
hidden_size=150
dropout_ratio=0.5
tune_emb=True

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(dropout_ratio))

# we add a Convolution1D, which will learn nb_filter (word group) filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, 
                        border_mode='valid', activation='relu', subsample_length=1))

# we use standard max pooling (halving the output of the previous layer):
model.add(MaxPooling1D(pool_length=pool_length))
model.add(Dropout(dropout_ratio))

# We flatten the output of the conv layer, so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_size))
model.add(Activation('relu'))
model.add(Dropout(dropout_ratio))

print('Doing classification with class #', nb_classes)
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

loss       = 'categorical_crossentropy'
class_mode = 'categorical'
optimizer = "adadelta"

model.compile(optimizer=optimizer, loss=loss,  class_mode=class_mode, metrics=['accuracy'])

Doing classification with class # 3


/usr/local/lib/python3.5/site-packages/keras/models.py:546: UserWarning: "class_mode" argument is deprecated, please remove it.
  warnings.warn('"class_mode" argument is deprecated, '


In [51]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

model_name = "test_dnn_twitter"
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
checkpointer = ModelCheckpoint(filepath=model_name, monitor='val_loss', verbose=1, save_best_only=True)

In [52]:
model.fit(x_train, y_train, batch_size=128, nb_epoch=25,
        validation_data=(x_val, y_val), show_accuracy=True, verbose=1, callbacks=[earlystopper, checkpointer])

print ("Loading ...", model_name)
model.load_weights(model_name)       
print("Test model ...")

/usr/local/lib/python3.5/site-packages/keras/models.py:610: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 6852 samples, validate on 2936 samples
Epoch 1/25
6852/6852 [==============================] - 25s - loss: 0.7804 - acc: 0.5410 - val_loss: 0.7266 - val_acc: 0.5470
Epoch 2/25
6852/6852 [==============================] - 25s - loss: 0.7230 - acc: 0.5544 - val_loss: 0.7135 - val_acc: 0.5490
Epoch 3/25
6852/6852 [==============================] - 25s - loss: 0.7083 - acc: 0.5617 - val_loss: 0.7053 - val_acc: 0.5484
Epoch 4/25
6852/6852 [==============================] - 31s - loss: 0.7024 - acc: 0.5641 - val_loss: 0.7049 - val_acc: 0.5484
Epoch 5/25
6852/6852 [==============================] - 26s - loss: 0.6994 - acc: 0.5747 - val_loss: 0.7007 - val_acc: 0.5484
Epoch 6/25
6852/6852 [==============================] - 27s - loss: 0.6963 - acc: 0.5757 - val_loss: 0.6987 - val_acc: 0.5460
Epoch 7/25
6852/6852 [==============================] - 27s - loss: 0.6957 - acc: 0.5784 - val_loss: 0.6986 - val_acc: 0.5484
Epoch 8/25
6852/6852 [==============================] - 26s - loss: 0.

In [ ]:
list(map(,corpus))

In [45]:
nb_validation_samples = int(0.3 * data.shape[0])
#labels = np_utils.to_categorical(numpy.asarray(corpus_target))
x_train = data[:-nb_validation_samples]
y_train = Y_train[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = Y_train[-nb_validation_samples:]

In [46]:
sequences_test = tokenizer.texts_to_sequences(docs_new)
#MAX_SEQUENCE_LENGTH = numpy.max(list(map(lambda x: len(x), sequences)))
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)

In [236]:
numpy.unique(y_pred)

array([1, 2])

In [245]:
y_pred

array([1, 2, 2, ..., 1, 1, 1])

In [235]:
#from sklearn.metrics import roc_curve, auc
from sklearn import metrics
y_pred = model.predict_classes(data_test)
y_test = numpy.array(encoded_test_Y)

acc2 = metrics.accuracy_score(y_test, y_pred)
print("Raw Accuracy:", acc2)

#get label ids in sorted
class_labels = sorted(label_id, key=label_id.get)
#print (class_labels)

print (metrics.classification_report(y_test, y_pred, target_names=class_labels, digits=4) )

print ("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred, labels=range(0, len(class_labels))))

1088/1088 [==============================] - 1s     
Raw Accuracy: 0.883272058824


NameError: name 'label_id' is not defined

In [176]:
#vt_sequence = TfidfVectorizer(min_df=1, tokenizer=tknzr.tokenize)
X_train = tokenizer.texts_to_sequences(corpus, mode='tfidf')
X_test = tokenizer.texts_to_sequences(docs_new, mode='tfidf')

TypeError: texts_to_sequences() got an unexpected keyword argument 'mode'

In [166]:
X_train.shape

(9788, 10000)

In [294]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

batch_size = 32
nb_epoch = 8
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(10000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    nb_epoch=nb_epoch, batch_size=batch_size,
                    verbose=1, validation_split=0.1)
score = model.evaluate(X_test, Y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Building model...


Exception: Error when checking model input: expected dense_input_1 to have shape (None, 10000) but got array with shape (9788, 1)

In [77]:
model.predict_classes(tokenizer.texts_to_matrix(boston[:100][boston.columns[1]], mode='tfidf'), batch_size=32, verbose=0)

array([1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2,
       1, 2, 1, 1, 2, 2, 2, 1])

In [184]:
#boston = pd.read_csv('./data/CrisisLexT26/2013_Boston_bombings/2013_Boston_bombings-tweets_labeled.csv')
boston = pd.read_csv('./data/CrisisLexT26/2012_Philipinnes_floods/2012_Philipinnes_floods-tweets_labeled.csv')

In [185]:
boston[:10]['Informativeness']

KeyError: 'Informativeness'

401
401


In [253]:
list(zip(map(lambda x: lbl_encoder.inverse_transform(x), y_pred),corpus_target,docs_new))

[('Not Relevant', 'Relevant', 'Just happened a terrible car crash'),
 ('Relevant',
  'Relevant',
  'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'),
 ('Relevant',
  'Relevant',
  'Heard about #earthquake is different cities, stay safe everyone.'),
 ('Relevant',
  'Relevant',
  'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all'),
 ('Relevant', 'Relevant', 'Forest fire near La Ronge Sask. Canada'),
 ('Relevant',
  'Relevant',
  "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"),
 ('Relevant',
  'Relevant',
  '13,000 people receive #wildfires evacuation orders in California '),
 ('Not Relevant',
  'Relevant',
  'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '),
 ('Relevant',
  'Relevant',
  '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #w

In [41]:
#embfile = open('/Users/pratheekrebala/Downloads/crisis_all_data_vecs.bin', 'rb')
embfile = open('./data/crisis_embeddings.text', 'r')
#embfile.readline()

In [42]:
embeddings_index = {}
f = embfile #open('/Users/pratheekrebala/Downloads/crisis_all_data_vecs.bin', 'rb')
for line in f:
    values = line.split()
    word = values[0]
    coefs = numpy.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 20733 word vectors.


In [103]:
def openemb(embfile):
    f = open(embfile, 'rb')

    vec_size_got = int ( f.readline().strip().split()[1]) # read the header to get vec dim

    if vec_size_got != vec_size:
        print " vector size provided and found in the file don't match!!!"
        raw_input(' ')
        exit(1)

    # load Embedding matrix
    row_nb = index_from+len(vocab_idmap)    
    E      = 0.01 * np.random.uniform( -1.0, 1.0, (row_nb, vec_size) )

    wrd_found = {}


    #fo = open("reduced_crisis_emb.txt", "rw+")
    #fo.write("300000 300\n")	
    #test = 0
    for line in f: # read from the emb file
        all_ent   = line.split()
	#print(all_ent)
        word, vec = all_ent[0].lower(), map (float, all_ent[1:])

        if vocab_idmap.has_key(word):
            wrd_found[word] = 1 
            wid    = vocab_idmap[word] + index_from
            E[wid] = np.array(vec)
	    #test = test + 1
	    #write to tmp file
	    #fo.write(line)
    #print(test)
    #fo.close()
    f.close()
    print " Number of words found in emb matrix: " + str (len (wrd_found)) + " of " + str (len(vocab_idmap))

    return E       

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-103-a2bb3733d6ca>, line 7)

In [44]:
boston.columns = list(map(lambda x: x.strip(),list(boston.columns)))

In [51]:
boston['Informativeness'] == 'Not applicable'

True

In [16]:
crowdflower[:1000]["text"].apply(tokenize_it.tweet_to_tokens)

0                 [just, happen, a, terribl, car, crash]
1      [our, deed, are, the, reason, of, thi, #, eart...
2      [heard, about, #, earth, quak, is, differ, cit...
3      [there, is, a, forest, fire, at, spot, pond, ,...
4        [forest, fire, near, la, rong, sask, ., canada]
5      [all, resid, ask, to, 'shelter, in, place', ar...
6      [|~num~|, peopl, receiv, #, wild, fire, evacu,...
7      [just, got, sent, thi, photo, from, rubi, #, a...
8      [#, fire, rocki, updat, |~happy~|, california,...
9        [apocalyps, light, ., #, spokan, #, wild, fire]
10     [#, flood, #, disast, heavi, rain, caus, flash...
11     [typhoon, soudelor, kill, |~num~|, in, china, ...
12                [we'r, shake, ..., it', an, earthquak]
13     [i'm, on, top, of, the, hill, and, i, can, see...
14     [there', an, emerg, evacu, happen, now, in, th...
15     [i'm, afraid, that, the, tornado, is, come, to...
16     [|~units~|, peopl, die, from, the, heat, wave,...
17     [haha, south, tampa, is,

In [13]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=10000, lower=True, split=" ")
#keras.preprocessing.text.text_to_word_sequence(crowdflower[200:202]['text'][200])

In [65]:
tokenizer.texts_to_sequences(texts=list(crowdflower['text']))

[[32, 748, 5, 1541, 123, 96],
 [112, 5394, 24, 4, 844, 8, 21, 252, 153, 1743, 3694, 87, 41],
 [373, 54, 252, 11, 1305, 1813, 664, 1499, 267],
 [76,
  11,
  5,
  167,
  44,
  20,
  833,
  2317,
  7010,
  24,
  3929,
  862,
  4,
  688,
  10,
  1296,
  476,
  100,
  41],
 [167, 44, 202, 930, 8526, 8345, 1442],
 [41,
  1563,
  1532,
  6,
  7721,
  7,
  5341,
  24,
  145,
  8795,
  19,
  1749,
  39,
  294,
  271,
  57,
  2185,
  7,
  783,
  1592,
  24,
  1240],
 [772, 2833, 61, 4323, 1086, 271, 1592, 7, 110],
 [32, 102, 1466, 21, 293, 23, 5425, 2319, 30, 291, 23, 1086, 7086, 71, 5, 201],
 [3004,
  422,
  110,
  1779,
  850,
  875,
  7,
  820,
  7591,
  392,
  6,
  1273,
  423,
  44,
  5255,
  1086],
 [390, 3187, 5750, 1086],
 [206, 77, 901, 310, 1056, 896, 299, 8, 1729, 7, 7726, 1124, 2754, 1675],
 [228, 785, 357, 2270, 7, 975, 9, 1289],
 [826, 3450, 65, 42, 252],
 [47, 12, 223, 8, 4, 2078, 9, 10, 72, 111, 5, 44, 7, 4, 4322],
 [478, 42, 75, 271, 1241, 56, 7, 4, 576, 862, 4, 688],
 [47, 2214

In [2]:
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, nb_epoch=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1)             101         lstm_1[0][0]                     
Total params: 213301
____________________________________________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 981s - loss: 0.6418 - acc: 0.6308   
Epoch 2/3
25000/25000 [==============================] - 1034s - loss: 0

In [206]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(nb_words=10000, test_split=0.2)

In [207]:
inv_map = {v: k for k, v in reuters.get_word_index().items()}
inv_map

{1: 'the',
 2: 'of',
 3: 'to',
 4: 'in',
 5: 'said',
 6: 'and',
 7: 'a',
 8: 'mln',
 9: '3',
 10: 'for',
 11: 'vs',
 12: 'dlrs',
 13: 'it',
 14: 'reuter',
 15: '000',
 16: '1',
 17: 'pct',
 18: 'on',
 19: 'from',
 20: 'is',
 21: 'that',
 22: 'its',
 23: 'cts',
 24: 'by',
 25: 'at',
 26: 'year',
 27: 'be',
 28: 'with',
 29: '2',
 30: 'will',
 31: 'was',
 32: 'billion',
 33: 'he',
 34: 'u',
 35: 's',
 36: 'net',
 37: 'has',
 38: 'would',
 39: 'an',
 40: 'as',
 41: '5',
 42: 'not',
 43: 'loss',
 44: '4',
 45: '1986',
 46: 'company',
 47: 'which',
 48: 'but',
 49: 'this',
 50: 'shr',
 51: 'last',
 52: 'are',
 53: 'lt',
 54: 'have',
 55: 'or',
 56: '6',
 57: 'bank',
 58: '7',
 59: 'were',
 60: '8',
 61: 'had',
 62: 'oil',
 63: 'trade',
 64: 'share',
 65: 'one',
 66: 'about',
 67: '0',
 68: 'inc',
 69: '9',
 70: 'new',
 71: 'profit',
 72: 'also',
 73: 'market',
 74: 'they',
 75: 'two',
 76: 'shares',
 77: 'stock',
 78: 'corp',
 79: 'tonnes',
 80: '10',
 81: 'up',
 82: 'been',
 83: 'revs',
 8

In [208]:
list(map(lambda x: inv_map[x], X_train[0])),X_train[0]

(['the',
  'of',
  'of',
  'mln',
  'loss',
  'for',
  'plc',
  'said',
  'at',
  'only',
  'ended',
  'said',
  'commonwealth',
  'could',
  '1',
  'traders',
  'now',
  'april',
  '0',
  'a',
  'after',
  'said',
  'from',
  '1985',
  'and',
  'from',
  'foreign',
  '000',
  'april',
  '0',
  'prices',
  'its',
  'account',
  'year',
  'a',
  'but',
  'in',
  'this',
  'mln',
  'home',
  'an',
  'states',
  'earlier',
  'and',
  'rise',
  'and',
  'revs',
  'vs',
  '000',
  'its',
  '16',
  'vs',
  '000',
  'a',
  'but',
  '3',
  'psbr',
  'oils',
  'several',
  'and',
  'shareholders',
  'and',
  'dividend',
  'vs',
  '000',
  'its',
  'all',
  '4',
  'vs',
  '000',
  '1',
  'mln',
  'agreed',
  'largely',
  'april',
  '0',
  'are',
  '2',
  'states',
  'will',
  'billion',
  'total',
  'and',
  'against',
  '000',
  'pct',
  'dlrs'],
 [1,
  2,
  2,
  8,
  43,
  10,
  447,
  5,
  25,
  207,
  270,
  5,
  3095,
  111,
  16,
  369,
  186,
  90,
  67,
  7,
  89,
  5,
  19,
  102,
  6,
